In [ ]:
import os
import json
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI
load_dotenv()

MODEL = 'gpt-4o-mini'
openai = OpenAI()

system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."


ticket_prices = {
    "london": 100,
    "paris": 150,
    "new york": 300,
    "tokyo": 500,
    "berlin": 200,
}

In [ ]:

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called with destination_city: {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")


def book_ticket(destination_city):
    print(f"Tool book_ticket called with destination_city: {destination_city}")
    city = destination_city.lower()
    if city in ticket_prices:
        return f"Ticket to {city.title()} booked successfully!"
    else:
        return "Booking failed. Unknown destination city."


price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticker price, for exampl when a customer asls: 'How much is a ticket to this city?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to."
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }   
}

book_function = {
    "name": "book_ticket",
    "description": "Book a ticket to the destination city. Call this function whenever you are asked to book a ticket.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to."
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{'type': 'function', 'function': price_function},
         {'type': 'function', 'function': book_function}]


def handle_tool_calls(message):
    print(f"Handling tool calls: {message.tool_calls}")
    responses = []
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get("destination_city")
        if tool_call.function.name == "book_ticket":
            res = book_ticket(city)
        elif tool_call.function.name == "get_ticket_price":
            res = get_ticket_price(city)

        
        response = {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps({
                "destination_city": city,
                "res": res
            })
        }

        responses.append(response)
    return responses



def chat(message, history):
    messages = [{'role': 'system', 'content': system_message}]
    for human, assistant in history:
        messages.append({'role': 'user', 'content': human})
        messages.append({'role': 'assistant', 'content': assistant})
    
    messages.append({'role': 'user', 'content': message})

    response = openai.chat.completions.create(
        model = MODEL,
        messages = messages,
        tools = tools
    )



    if response.choices[0].finish_reason == "tool_calls":
        assistant_msg = response.choices[0].message
        tool_responses = handle_tool_calls(assistant_msg)
        messages.append({
            "role": "assistant",
            "tool_calls": assistant_msg.tool_calls,
            "content": assistant_msg.content
        })
        
        messages.extend(tool_responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    return response.choices[0].message.content



gr.ChatInterface(fn=chat).launch()

In [ ]:
import base64
from io import BytesIO
from PIL import Image


def artist(cities):
    image_response = openai.images.generate(
        model="dall-e-3",
        prompt=f"An image representing a vacation in the following list of cities: {cities}, showing tourist spots and everything unique about the cities: {cities}, in a vibrant pop-art style",
        size="1024x1024",
        response_format="b64_json"
    )

    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
image = artist("[London]")
display(image)

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message
    )

    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [ ]:
talker("Well, hi there")

In [ ]:
def handle_tool_calls(message):
    print(f"Handling tool calls: {message.tool_calls}")

    responses = []
    cities = []

    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get("destination_city")
        if tool_call.function.name == "book_ticket":
            res = book_ticket(city)
        elif tool_call.function.name == "get_ticket_price":
            res = get_ticket_price(city)

        
        response = {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": json.dumps({
                "destination_city": city,
                "res": res
            })
        }

        responses.append(response)
        cities.append(city)
        
    return responses, cities

In [ ]:
def translate_to_spanish(text):
    system_message = "You are a helpful assistant for an Airline called FlightAI. "
    system_message += "Translate the given text to Spanish. Do not add any additional information, just the translation."

    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": text}
        ]
    )

    return response.choices[0].message.content

In [ ]:
import random

def chat(message, history):

    image = None
    conversation = [{'role': 'system', 'content': system_message}]

    for human, assistant in history:
        conversation.append({'role': 'user', 'content': human})
        conversation.append({'role': 'assistant', 'content': assistant})
    conversation.append({'role': 'user', 'content': message})
    response = openai.chat.completions.create(model=MODEL, messages=conversation, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = tool_call = response.choices[0].message
        tool_responses, cities = handle_tool_calls(message)
        conversation.append({
            "role": "assistant",
            "tool_calls": message.tool_calls,
            "content": message.content
        })

        conversation.extend(tool_responses)

        random_city = random.choice(cities)

        image = artist(str(cities))

        response = openai.chat.completions.create(model=MODEL, messages=conversation, tools=tools)
    
    reply = response.choices[0].message.content
    talker(reply)

    return reply, image

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500)
        image_output = gr.Image(height=500)
    
    with gr.Row():
        msg = gr.Textbox(label="Chat with our AI assistant")
    
    with gr.Row():
        clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [(user_message, None)]
    
    def bot(history):
        user_message = history[-1][0]
        bot_message, image = chat(user_message, history[:-1])
        history[-1][1] = bot_message
        return history, image
    
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, [chatbot, image_output])
    
    clear.click(lambda: None, None, chatbot, queue=False)

ui.launch()

In [ ]:
import os
import uuid
import numpy as np
from scipy.io.wavfile import write as write_audio

def save_audio_file(audio, history):
    try:
        if audio is None:
            return "No audio recorded."

        sample_rate, data = audio

        filename = f"recording.mp3"
        save_path = os.path.join("recordings", filename)

        os.makedirs("recordings", exist_ok=True)

        write_audio(save_path, sample_rate, data.astype(np.int16))

        audio_file = open('/Users/slavacalestru/Desktop/DonerLLMEngineering_NF/New/week2/recordings/recording.mp3', "rb")

        transcription = openai.audio.transcriptions.create(
            model="whisper-1", 
            file=audio_file
        )

        print(transcription.text)


        return "Audio saved and transcribed.", history + [[transcription.text, None]]

    
    except Exception as e:
        return history + [[f"[Error] {e}", None]], f"Failed to save audio: {e}"

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500)
        image_output = gr.Image(height=500)
        with gr.Column():
            spanish_user_input = gr.Textbox(label="Spanish User Input", interactive=False)
            spanish_llm_output = gr.Textbox(label="Spanish LLM Output", interactive=False)
    
    with gr.Row():
        msg = gr.Textbox(label="Chat with our AI assistant")
    
    with gr.Row():
        clear = gr.Button("Clear")

    with gr.Row():
        audio_input = gr.Audio(sources=["microphone"], label="Record your voice")
        save_status = gr.Textbox(label="Audio Save Status", interactive=False)
        save_button = gr.Button("Save Audio")

    def user(user_message, history):
        return "", history + [(user_message, None)]
    
    def bot(history):
        user_message = history[-1][0]
        print(user_message)
        
        bot_message, image = chat(user_message, history[:-1])
        history[-1][1] = bot_message

        spanish_user_translation = translate_to_spanish(user_message)
        spanish_llm_translation = translate_to_spanish(bot_message)

        return history, image, spanish_user_translation, spanish_llm_translation
    
    save_button.click(
        fn=save_audio_file, 
        inputs=[audio_input, chatbot], 
        outputs=[chatbot, save_status]    ).then(
            fn=bot,
            inputs=chatbot,
            outputs=[chatbot, image_output, spanish_user_input, spanish_llm_output]
        )

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, [chatbot, image_output, spanish_user_input, spanish_llm_output])
    
    clear.click(lambda: None, None, chatbot, queue=False)

    

ui.launch()

In [ ]:
! =pip install https://git.ffmpeg.org/ffmpeg.git